In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
import numpy as np

2024-07-13 13:17:44.930179: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 13:17:44.990822: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-13 13:17:44.992293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 13:17:46.040645: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Sample data
texts = [
		"I love natural language processing.",
		"Natural language processing is fascinating.",
		"NLP is a subfield of artificial intelligence.",
		"I dislike bugs in my code.",
		"Sentiment analysis is not always easy.",
        "Feel terrible to work on weekend"
	]

labels = [1, 1, 1, 0, 0, 0]  # 1 for positive, 0 for negative

In [3]:
# Tokenize and pad sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts) # creates index of the texts word 
sequences = tokenizer.texts_to_sequences(texts)
## Now, we can use texts_to_sequences() to convert the texts into sequences:
## The output will be a list of sequences, where each sequence represents a text converted into a sequence of integers based on the tokenizer's word index:
## The first text "I love machine learning" is converted to the sequence [1, 2, 3, 4].
## The second text "Deep learning is amazing" is converted to the sequence [5, 6, 7, 8].

In [4]:
print(sequences)

[[2, 6, 3, 4, 5], [3, 4, 5, 1, 7], [8, 1, 9, 10, 11, 12, 13], [2, 14, 15, 16, 17, 18], [19, 20, 1, 21, 22, 23], [24, 25, 26, 27, 28, 29]]


In [5]:
print(tokenizer.word_index)

{'is': 1, 'i': 2, 'natural': 3, 'language': 4, 'processing': 5, 'love': 6, 'fascinating': 7, 'nlp': 8, 'a': 9, 'subfield': 10, 'of': 11, 'artificial': 12, 'intelligence': 13, 'dislike': 14, 'bugs': 15, 'in': 16, 'my': 17, 'code': 18, 'sentiment': 19, 'analysis': 20, 'not': 21, 'always': 22, 'easy': 23, 'feel': 24, 'terrible': 25, 'to': 26, 'work': 27, 'on': 28, 'weekend': 29}


In [6]:
X = pad_sequences(sequences)
labels = np.array(labels)

In [7]:
print(X)

[[ 0  0  2  6  3  4  5]
 [ 0  0  3  4  5  1  7]
 [ 8  1  9 10 11 12 13]
 [ 0  2 14 15 16 17 18]
 [ 0 19 20  1 21 22 23]
 [ 0 24 25 26 27 28 29]]


In [8]:
print(type(X))
print(type(labels))
print(X.shape)
print(labels.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(6, 7)
(6,)


In [9]:
# Build Bag of Words model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=X.shape[1]))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, labels, epochs=40, batch_size=32) # Train the model

Epoch 1/40
1/1 [==============================] - 1s 926ms/step - loss: 0.6946 - accuracy: 0.5000
Epoch 2/40
1/1 [==============================] - 0s 8ms/step - loss: 0.6850 - accuracy: 1.0000
Epoch 3/40
1/1 [==============================] - 0s 7ms/step - loss: 0.6775 - accuracy: 1.0000
Epoch 4/40
1/1 [==============================] - 0s 7ms/step - loss: 0.6696 - accuracy: 1.0000
Epoch 5/40
1/1 [==============================] - 0s 6ms/step - loss: 0.6607 - accuracy: 1.0000
Epoch 6/40
1/1 [==============================] - 0s 6ms/step - loss: 0.6504 - accuracy: 1.0000
Epoch 7/40
1/1 [==============================] - 0s 6ms/step - loss: 0.6388 - accuracy: 1.0000
Epoch 8/40
1/1 [==============================] - 0s 6ms/step - loss: 0.6256 - accuracy: 1.0000
Epoch 9/40
1/1 [==============================] - 0s 7ms/step - loss: 0.6109 - accuracy: 1.0000
Epoch 10/40
1/1 [==============================] - 0s 7ms/step - loss: 0.5944 - accuracy: 1.0000
Epoch 11/40
1/1 [====================

In [10]:
model.save('models/sentiment_model.h5') # Save the model

/home/ggupte/practice/ds/deep_learning/venv-deep-learn/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
loaded_model = tf.keras.models.load_model('models/sentiment_model.h5') # Load the model

In [12]:
# Use the loaded model for predictions on new data
new_texts = ["I enjoy working with natural language processing.", "This movie is terrible."]

In [13]:
# Tokenize and pad the new texts
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_data = pad_sequences(new_sequences, maxlen=X.shape[1])

In [14]:
# Make predictions
predictions = loaded_model.predict(new_data)
print(predictions)

1/1 [==============================] - 0s 92ms/step
[[0.9878718]
 [0.8670116]]


In [15]:
# Convert the predictions to labels (0 for negative, 1 for positive)
predicted_labels = (predictions > 0.5).astype(int)

In [16]:
# Display the results
for text, label in zip(new_texts, predicted_labels):
	sentiment = "Positive" if label == 1 else "Negative"
	print(f"Text: {text} | Sentiment: {sentiment}")

Text: I enjoy working with natural language processing. | Sentiment: Positive
Text: This movie is terrible. | Sentiment: Positive


text_to_sequences Function:
	Input:
	texts: This is the input text data that you want to convert to sequences. It can be a list of strings, where each string represents a piece of text.
	Output:
	sequences: The output of text_to_sequences is a list of lists. Each inner list corresponds to a sequence of integer indices representing the words in the corresponding input text. The indices are assigned based on the vocabulary learned by the Tokenizer.

pad_sequences Function:
	Input:
	sequences: This is the list of lists obtained from the text_to_sequences function. Each inner list represents a sequence of integer indices.
	maxlen (optional): This is an integer, the maximum length of all sequences. If provided, sequences longer than maxlen will be truncated, and sequences shorter than maxlen will be padded.
	padding (optional): String, 'pre' or 'post'. If not provided, the default is 'pre'. If 'pre', padding is added at the beginning of the sequences; if 'post', padding is added at the end.
	truncating (optional): String, 'pre' or 'post'. If not provided, the default is 'pre'. If 'pre', sequences longer than maxlen are truncated at the beginning; if 'post', sequences longer than maxlen are truncated at the end.
	value (optional): Float or String, the padding value. If not provided, the default is 0.0.
	Output:
	padded_sequences: The output of pad_sequences is a 2D numpy array where each row represents a padded or truncated sequence of integer indices. The array can be directly used as input to a neural network.		